# Python-C interoperability

Python has an [excellent C API](https://docs.python.org/3/c-api/index.html) for writing extension modules in C/C++. Let's not use it.

In [10]:
with open("tmp.c", "w") as file:
    file.write("""
#include <math.h>
void runs_in_c(int N, double* p, double* px, double* py, double* pz) {
    int i;
    for (i = 0;  i < N;  i++) {
        p[i] = sqrt(px[i]*px[i] + py[i]*py[i] + pz[i]*pz[i]);
    }
}""")
import os
assert os.system("gcc -fPIC -shared tmp.c -o libtmp.so") == 0

In [11]:
import ctypes
libtmp = ctypes.cdll.LoadLibrary("./libtmp.so")

In [12]:
libtmp.runs_in_c.argtypes = (ctypes.c_int, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p)
libtmp.runs_in_c.restype = None

In [13]:
import numpy
px = numpy.random.normal(0, 10, 1000000)
py = numpy.random.normal(0, 10, 1000000)
pz = numpy.random.normal(0, 10, 1000000)
p = numpy.zeros(1000000)

libtmp.runs_in_c(1000000, p.ctypes.data, px.ctypes.data, py.ctypes.data, pz.ctypes.data)

In [14]:
p

array([12.71839662, 10.39316839, 18.87639032, ..., 21.49520311,
       19.42239314, 12.90649731])

**Going the other way:** running Python in C.

The GNU Scientific Library is a large collection of C functions, some of which take functions as arguments:

```c
struct gsl_function {
    double (*function) (double x, void* params);
    void* params;
};
```

Let's convince GSL to take a Python function as though it were a C function (in its derivative-calculating method).

In [6]:
def f(x, params):
    out = 3*x**2 + 2*x + 1
    print("f({}) = {}".format(x, out))
    return out

In [7]:
# express the (double, double -> void) function type in ctypes
func_type = ctypes.CFUNCTYPE(ctypes.c_double, ctypes.c_double, ctypes.c_void_p)

# express the gsl_function struct in ctypes
class gsl_function(ctypes.Structure):
    _fields_ = [("f", func_type), ("params", ctypes.c_void_p)]

# make a ctypes object for the function
callback = gsl_function(func_type(f), 0)

In [8]:
gslcblas = ctypes.CDLL("libgslcblas.so", mode=ctypes.RTLD_GLOBAL)
gsl = ctypes.CDLL("libgsl.so")

# pointers to GSL's output: result and abserr
p_result = ctypes.POINTER(ctypes.c_double)(ctypes.c_double(-1))
p_abserr = ctypes.POINTER(ctypes.c_double)(ctypes.c_double(-1))

# call the function
assert gsl.gsl_deriv_central(
    ctypes.POINTER(gsl_function)(callback), ctypes.c_double(2), ctypes.c_double(1e-8), p_result, p_abserr) == 0

OSError: libgslcblas.so: cannot open shared object file: No such file or directory

In [9]:
p_result[0], p_abserr[0]

NameError: name 'p_result' is not defined

**Doing it in ROOT:** ROOT has a built-in compiler and PyROOT automatically generates bindings, making it almost a one-liner.

In [1]:
import ROOT

Welcome to JupyROOT 6.14/00


In [2]:
ROOT.gInterpreter.Declare("""
void runs_in_cpp(int N, double* p, double* px, double* py, double* pz) {
    int i;
    for (i = 0;  i < N;  i++) {
        p[i] = sqrt(px[i]*px[i] + py[i]*py[i] + pz[i]*pz[i]);
    }
}""")

True

In [3]:
import numpy
px = numpy.random.normal(0, 10, 1000000)
py = numpy.random.normal(0, 10, 1000000)
pz = numpy.random.normal(0, 10, 1000000)
p = numpy.zeros(1000000)

ROOT.runs_in_cpp(1000000, p, px, py, pz)

In [4]:
p

array([15.80089086, 19.30883965, 14.48536631, ..., 22.93886971,
       24.7736794 , 12.29306962])

**To mix Python objects with C++ objects, use Cython.**

Cython (not CPython!) is a halfway language between Python and C++, with constructs familiar to both.

It's usually used as to make libraries (root_numpy, rootpy, NumPythia, pyjet all use it), but I think its Jupyter extension is a killer app.

In [6]:
%load_ext Cython

*(continued on next page)*

In [7]:
%%cython -a --cplus -c-std=c++11 -c-O3

import numpy

cdef extern from *:
    """
void pure_cpp(int N, double* p, double* px, double* py, double* pz) {
    for (int i = 0;  i < N;  i++) {
        p[i] = sqrt(px[i]*px[i] + py[i]*py[i] + pz[i]*pz[i]);
    }
}"""
    void pure_cpp(int N, double* p, double* px, double* py, double* pz)

def mixed_python_cpp(px, py, pz):
    if not isinstance(px, numpy.ndarray) or not isinstance(px, numpy.ndarray) or not isinstance(px, numpy.ndarray):
        raise TypeError("should be Numpy arrays")
    cdef int N = len(px)
    p = numpy.empty(N)
    cdef double* p_ptr = <double*>(<size_t>p.ctypes.data)
    cdef double* px_ptr = <double*>(<size_t>px.ctypes.data)
    cdef double* py_ptr = <double*>(<size_t>py.ctypes.data)
    cdef double* pz_ptr = <double*>(<size_t>pz.ctypes.data)
    pure_cpp(N, p_ptr, px_ptr, py_ptr, pz_ptr)
    return p

cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++


In [8]:
import numpy
px = numpy.random.normal(0, 10, 1000000)
py = numpy.random.normal(0, 10, 1000000)
pz = numpy.random.normal(0, 10, 1000000)

mixed_python_cpp(px, py, pz)

array([10.59823266, 22.73848972, 13.11037692, ..., 18.2356867 ,
       11.63217723, 19.98634378])

# PyCUDA (and PyOpenCL): arbitrary GPU programming

The lowest of low-level programming— programming and running accelerator devices— can be done from the comfort of a notebook.

In [ ]:
import pycuda
import pycuda.autoinit
import pycuda.driver
import pycuda.compiler

compiled_cuda = pycuda.compiler.SourceModule("""
__global__ void runs_on_gpu(float* p, float* px, float* py, float* pz) {
    const int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < 1000000) {
        p[i] = sqrt(px[i]*px[i] + py[i]*py[i] + pz[i]*pz[i]);
    }
}""")
runs_on_gpu = compiled_cuda.get_function("runs_on_gpu")

In [ ]:
import numpy

px = pycuda.driver.In(numpy.random.normal(0, 10, 1000000).astype(numpy.float32))
py = pycuda.driver.In(numpy.random.normal(0, 10, 1000000).astype(dtype=numpy.float32))
pz = pycuda.driver.In(numpy.random.normal(0, 100, 1000000).astype(dtype=numpy.float32))
p = numpy.zeros(1000000, dtype=numpy.float32)

runs_on_gpu(pycuda.driver.Out(p), px, py, pz, block=(1024, 1, 1), grid=(1000000 // 1024 + 1, 1))

In [ ]:
p

Working with Numpy arrays is already GPU-like.

In [ ]:
import pycuda.gpuarray

px = pycuda.gpuarray.to_gpu(numpy.random.normal(0, 10, 1000000))
py = pycuda.gpuarray.to_gpu(numpy.random.normal(0, 10, 1000000))
pz = pycuda.gpuarray.to_gpu(numpy.random.normal(0, 100, 1000000))

type(px)

In [ ]:
p = (px**2 + py**2 + pz**2)**0.5
type(p)

In [ ]:
p.get()

Isn't that what CuPy does? Yes, yes it is.

Once, there was also a "gnumpy" (U Toronto CS dept). Like Numeric and numarray, this is an active area of development.

# Numba-CUDA: compile *Python* code on the GPU

Numba can compile a subset of Python for CPUs with LLVM or GPUs with CUDA.

In [ ]:
import math
import numba.cuda

@numba.cuda.jit
def runs_on_gpu(p, px, py, pz):
    i = numba.cuda.grid(1)
    p[i] = math.sqrt(px[i]**2 + py[i]**2 + pz[i]**2)

In [ ]:
px = numpy.random.normal(0, 10, 1000000)
py = numpy.random.normal(0, 10, 1000000)
pz = numpy.random.normal(0, 100, 1000000)
p = numpy.zeros(1000000)

runs_on_gpu(p, px, py, pz)

In [ ]:
p

# The snake eating its own tail: Numpy+ctypes to hack Python

Beyond its use for doing math, Numpy is a tool for interpreting arrays from other sources.

In [ ]:
import ctypes
import numpy

jemalloc = ctypes.cdll.LoadLibrary("libjemalloc.so")   # a specialized memory-allocator
jemalloc.malloc.argtypes = (ctypes.c_size_t,)
jemalloc.malloc.restype = ctypes.POINTER(ctypes.c_double)

In [ ]:
ptr = jemalloc.malloc(1000 * numpy.dtype(numpy.float64).itemsize)
ptr

In [ ]:
ctypes.addressof(ptr.contents)

In [ ]:
ptr.__array_interface__ = {
    "version": 3,
    "typestr": numpy.ctypeslib._dtype(type(ptr.contents)).str,
    "data": (ctypes.addressof(ptr.contents), False),
    "shape": (1000 * numpy.dtype(numpy.float64).itemsize,)
}
array = numpy.array(ptr, copy=False)
array[:] = 3.14
array

**Think about what this means:** we can cast any region in memory as a Numpy array and make arbitrary changes to it.

<center><img src="https://vignette.wikia.nocookie.net/villains/images/a/a4/Evil_Calvin.jpeg/revision/latest?cb=20131009203849&format=original"></center>

In [ ]:
hello = b"Hello, world!"

Strings and byte-strings are immutable in Python; a fact Python uses for optimization.

In [ ]:
hello[4:8] = "????"

But if we cast it as a Numpy array and make it writeable...

In [ ]:
array = numpy.frombuffer(hello, dtype=numpy.uint8)
array

In [ ]:
array.flags.writeable = True
array[4:8] = [69, 86, 73, 76]

In [ ]:
hello

We can use Python's C API without using C.

In [ ]:
# C struct of a Python object, represented with ctypes
class PyObject(ctypes.Structure): pass
PyObject._fields_ = [("ob_refcnt", ctypes.c_size_t), ("ob_type", ctypes.POINTER(PyObject))]

The above is equivalent to

```c
struct PyObject {
    size_t ob_refcnt;
    PyObject* ob_type;
    // there's more, but we don't need to encode it...
};
```

Not guaranteed in all implementations, but in CPython, `id(obj)` is a C pointer to `obj`.

In [ ]:
hello_ptr = PyObject.from_address(id(hello))
hello_ptr

In [ ]:
hello_ptr.__array_interface__ = {
    "version": 3,
    "typestr": "u1",
    "data": (ctypes.addressof(hello_ptr), False),
    "shape": (100,)
}
array = numpy.array(hello_ptr, copy=False)
array.tobytes()

The `ob_refcnt` is Python's internal reference count, used by the garbage collector.

In [ ]:
hello_ptr.ob_refcnt

In [ ]:
import sys
sys.getrefcount(hello)   # overcounts by 1 because it's passed as an argument to this function

In [ ]:
another_reference = hello

In [ ]:
hello_ptr.ob_refcnt, sys.getrefcount(hello)

Now try something truly sinister: can we change the type of the object by setting its pointer?

In [ ]:
class DifferentClass(bytes):
    def has_a_method(self):
        return self * 3

In [ ]:
# change the class of this object by directly manipulating its pointer
hello_ptr.ob_type = ctypes.POINTER(PyObject)(PyObject.from_address(id(DifferentClass)))

In [ ]:
type(hello)

In [ ]:
hello.has_a_method()

# Dead kernel: the end!